데이터 다운로드

In [1]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 809M/814M [00:10<00:00, 85.8MB/s]
100% 814M/814M [00:10<00:00, 78.1MB/s]


압축 풀기

In [2]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [3]:
!unzip -q train.zip -d .

데이터셋 준비

In [4]:
import os
import tensorflow as tf
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


# 모델 생성

inception_v3 weight 파일 다운로드

In [5]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

inceptionV3 모델 불러와서 파라미터 설정하기

In [7]:
'''
# input_shape=(150, 150, 3)
  쉐입 변경
# include_top=False
  출력 레이어를 포함하지 않고 가져 오겠다는 뜻
# weights=None
  가중치를 가져오지 않겠다는 뜻
  파일을 따로 불러 올 것이기 때문에.
inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)

# 가중치 파일 불러오기
inception_model.load_weights('inception_v3.h5')


'''

from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
inception_model.load_weights('inception_v3.h5')

inception_model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 150, 150, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 74, 74, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 74, 74, 32)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 74, 74, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 72, 72, 32)     │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 72, 72, 32)     │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 72, 72, 32)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 72, 72, 64)     │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 72, 72, 64)     │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 72, 72, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 35, 35, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 35, 35, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 35, 35, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 35, 35, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 33, 33, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 33, 33, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

weight 업데이트 금지 (학습금지)

In [10]:
'''
for i in inception_model.layers:
  i.trainable=False

  레이어를 한 개씩 불러와서 트레이닝 하지 않게 함.
'''


from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
inception_model.load_weights('inception_v3.h5')

for i in inception_model.layers:
  i.trainable=False

출력층을 제외한 마지막레이어 추출해보기

In [12]:
'''
# 추출할 레이어 get_layer()로 불러오기
마지막레이어 = inception_model.get_layer('mixed7')

# 추출한 레이어 확인하기
print(마지막레이어)
print(마지막레이어.output)

'''

from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
inception_model.load_weights('inception_v3.h5')

for i in inception_model.layers:
  i.trainable=False

마지막레이어 = inception_model.get_layer('mixed7')

print(마지막레이어)
print(마지막레이어.output)

<Concatenate name=mixed7, built=True>
<KerasTensor shape=(None, 7, 7, 768), dtype=float32, sparse=False, name=keras_tensor_1783>


마지막레이어와 내가 만든 레이어 연결하기

In [ ]:
'''
layer1 = tf.keras.layers.Flatten()(마지막레이어.output)

특정 레이어만 가져오기 위해서 .output을 붙임.
'''

import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(drop1)

model = tf.keras.Model(inception_model.input, layer3)

# 모델 학습 준비(컴파일)

In [ ]:
'''
model = tf.keras.Model(inception_model.input, layer3)

인풋과 아웃풋을 명시해 줘야 함.


'''

import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(drop1)

model = tf.keras.Model(inception_model.input, layer3)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습

In [13]:
'''
model = tf.keras.Model(inception_model.input, layer3)

인풋과 아웃풋을 명시해줘야 함.

인풋을 'inception_model.input'처럼 한 이유는 layer3처럼 변수명에 레이어를 저장하지 않았기 때문에.

변수명에 저장하지 않은 모델 내의 input 레이어를 인풋으로 사용하려면, 이와 같이 해야 함.
'''


import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(drop1)

model = tf.keras.Model(inception_model.input, layer3)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 1553s 5s/step - accuracy: 0.9327 - loss: 0.3366 - val_accuracy: 0.9686 - val_loss: 0.0798
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 1508s 5s/step - accuracy: 0.9838 - loss: 0.0438 - val_accuracy: 0.9664 - val_loss: 0.0950


# 일부 레이어 학습시키기 (파인튜닝 하는법)

In [16]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
inception_model.load_weights('inception_v3.h5')

for i in inception_model.layers:
  i.trainable=False

unfreeze = False
for i in inception_model.layers:
  if i.name == 'mixed6':
    unfreeze = True
  if unfreeze == True:
    i.trainable = True

마지막레이어 = inception_model.get_layer('mixed7')

import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(drop1)

model = tf.keras.Model(inception_model.input, layer3)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 1775s 6s/step - accuracy: 0.9358 - loss: 0.1509 - val_accuracy: 0.9776 - val_loss: 0.0598
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 1713s 5s/step - accuracy: 0.9922 - loss: 0.0220 - val_accuracy: 0.9714 - val_loss: 0.0884
